In [1]:
import os
%pwd

'/Users/bansikumar_viradiya/Projects/Chicken-disease-classification-DL-project/research'

In [2]:
os.chdir("../")

In [3]:
%pwd


'/Users/bansikumar_viradiya/Projects/Chicken-disease-classification-DL-project'

In [5]:
# update the entity : return type of function if we want to create a custom returntype

from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir:Path
    source_url:str
    local_data_file:Path
    unzip_dir:Path


In [19]:
from src.CNNClassifier.constants import *
from src.CNNClassifier.utils.common import read_yaml, create_directories


In [23]:
class ConfigurationManager:
    def __init__(self, config_filepath = CONFIG_FILE_PATH, params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_url = config.source_url,
            local_data_file = config.local_data_file,
            unzip_dir = config.unzip_dir
        )

        return data_ingestion_config

In [18]:
import os
import urllib.request as request
import zipfile
from src.CNNClassifier import logger
from src.CNNClassifier.utils.common import get_size

In [20]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_url,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download completed!!")
        else:
            logger.info(f"File already exist")


    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [26]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-11-24 15:14:32,910: common: yaml_file : config/config.yaml loaded successfully]
[2025-11-24 15:14:32,912: common: yaml_file : params.yaml loaded successfully]
[2025-11-24 15:14:32,912: common: created the directory at : artifacts]
[2025-11-24 15:14:32,912: common: created the directory at : artifacts/data_ingestion]
[2025-11-24 15:14:34,598: 4033178932: artifacts/data_ingestion/data.zip download completed!!]
